In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install openslides

     |████████████████████████████████| 7.8MB 14.2MB/s 
     |████████████████████████████████| 409kB 37.9MB/s 
     |████████████████████████████████| 7.5MB 56.7MB/s 
     |████████████████████████████████| 921kB 49.3MB/s 
     |████████████████████████████████| 61kB 8.8MB/s 
     |████████████████████████████████| 1.8MB 50.5MB/s 
     |████████████████████████████████| 81kB 11.3MB/s 
     |████████████████████████████████| 583kB 47.8MB/s 
     |████████████████████████████████| 81kB 10.7MB/s 
     |████████████████████████████████| 3.1MB 51.8MB/s 
     |████████████████████████████████| 61kB 9.7MB/s 
     |████████████████████████████████| 235kB 58.5MB/s 
     |████████████████████████████████| 61kB 9.4MB/s 
     |████████████████████████████████| 2.7MB 51.0MB/s 
  Created wheel for openslides: filename=openslides-3.1-cp36-none-any.whl size=8198113 sha256=bff0af61a2e33c2610e7665945538dd4f555a4671c85318812e88c03fe26b949
  Stored in directory: /root/.cache/pip/wheels/56/f0/8d/0d72d1e96

In [ ]:
!apt update && apt install -y openslide-tools

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:13 http://security.ubuntu.com/ubu

In [ ]:
!pip install openslide-python

     |████████████████████████████████| 317kB 10.9MB/s 
  Created wheel for openslide-python: filename=openslide_python-1.1.1-cp36-cp36m-linux_x86_64.whl size=27365 sha256=78b809a005672f5f191c434cb0e876334916fe55e3958f13cebeb2834feb2c88
  Stored in directory: /root/.cache/pip/wheels/56/44/7e/16c9fc72cfbf1bffe48676b6835843d21abcc56566e958e7d6
Successfully built openslide-python


In [ ]:
from zipfile import ZipFile
file_name = "/content/drive/My Drive/CAMELYON17/training/center_0/patient_000.zip"
with ZipFile(file_name,'r') as zip :
  zip.extractall()
  print('finish')

finish


In [ ]:
import os
import sys
import random
import numpy as np
import openslide
from xml.etree.ElementTree import parse
import cv2
from PIL import Image

import pickle
import time

# for multiprocessing
import multiprocessing
from multiprocessing import Pool, Queue, Process, Array
from itertools import repeat

# user define variable
from user_define import Config as cf
from user_define import Hyperparams as hp

import pdb


class CAMELYON_PREPRO():
    """
    CAMELYON Dataset preprocessed by DEEPBIO
    Args:
        slide_filename (string) ex) 'b_0'
    """

    # config
    level = cf.level_for_preprocessing

    # hyper parameters
    patch_size = hp.patch_size
    num_of_patch = hp.number_of_patch_per_slide
    ratio_of_tumor_patch = hp.ratio_of_tumor_patch
    threshold_of_tumor_rate = hp.threshold_of_tumor_rate

    def __init__(self, usage, slide_filename):
        print("allocator", slide_filename)
        if usage != 'test':
            target_slide_path = os.path.join(cf.path_of_slide,
                                             slide_filename + '.tif')
            self.slide = openslide.OpenSlide(target_slide_path)
            self.downsamples = int(self.slide.level_downsamples[self.level])

            xml_filename = slide_filename + ".xml"
            target_xml_path = os.path.join(cf.path_of_annotation,
                                           xml_filename)
            self.annotation = self.get_annotation_from_xml(target_xml_path)

            # for save image
            self.patch_path = os.path.join(cf.path_for_result,
                                           slide_filename,
                                           cf.base_folder_for_patch)
            self.check_path(self.patch_path)

            self.etc_path = os.path.join(cf.path_for_result,
                                         slide_filename,
                                         cf.base_folder_for_etc)
            self.check_path(self.etc_path)

            # for create patch array
            self.tissue_mask = self.create_tissue_mask(cf.save_tissue_mask_image)
            self.tumor_mask = self.create_tumor_mask(cf.save_tumor_mask_image)

            num_of_patch_in_tumor = int(self.num_of_patch * self.ratio_of_tumor_patch)
            num_of_patch_in_tissue = self.num_of_patch - num_of_patch_in_tumor

            if usage == 'train':
                dila_of_tumor, ero_of_tumor = self.get_dilaero(self.tumor_mask)
                dila_of_tissue, _ = self.get_dilaero(self.tissue_mask)
                set_of_inform_in_tumor = self.get_inform_of_random_samples(
                                            ero_of_tumor,
                                            num_of_patch_in_tumor)
                set_of_inform_in_tissue = self.get_inform_of_random_samples(
                                            dila_of_tissue - dila_of_tumor,
                                            num_of_patch_in_tissue)

            elif usage == 'val':
                dila_of_tissue, _ = self.get_dilaero(self.tissue_mask)
                set_of_inform_in_tumor = self.get_inform_of_random_samples(
                                            self.tumor_mask,
                                            num_of_patch_in_tumor)
                set_of_inform_in_tissue = self.get_inform_of_random_samples(
                                            dila_of_tissue - self.tumor_mask,
                                            num_of_patch_in_tissue)

            elif usage == 'train_incorrect' or usage == 'val_incorrect':
                predict_filename = slide_filename + "_result.png"
                target_pred_path = os.path.join(cf.path_for_result,
                                                slide_filename,
                                                cf.base_folder_for_patch,
                                                predict_filename,
                                                )
                predict_array = cv2.imread(target_pred_path, 0)
                set_of_inform_in_tumor = self.get_inform_of_random_samples(
                                            (predict_array - self.tumor_mask),
                                            num_of_patch_in_tumor)
                set_of_inform_in_tissue = self.get_inform_of_random_samples(
                                            self.tissue_mask,
                                            num_of_patch_in_tissue)

            else:
                raise RuntimeError("usage is invalid value")

            self.set_of_inform = set_of_inform_in_tumor + set_of_inform_in_tissue
            self.set_of_inform = np.array(self.set_of_inform)

            self.set_of_patch = self.get_patch_data(cf.save_patch_images)
            self.set_of_patch = np.array(self.set_of_patch)

            if cf.save_thumbnail_image:
                self.thumbnail = self.create_thumbnail()
                self.draw_tumor_pos_on_thumbnail()
                self.draw_patch_pos_on_thumbnail()

        else :
            file_list = os.listdir(cf.path_of_task_1)
            file_list.sort()
            set_of_patch = []
            i = 0
            # for fn in tqdm(file_list):
            for fn in file_list:
                fp = os.path.join(cf.path_of_task_1, fn)
                img = cv2.imread(fp, cv2.IMREAD_COLOR)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                set_of_patch.append(img)
                i = i + 1
                print("\r%d" % (i), end="")
            print("\n")

            self.set_of_inform = np.array(file_list)
            self.set_of_patch = np.array(set_of_patch)

        self.create_dataset(usage, slide_filename)

    """
    param :
    return : annotations (list of numpy)
    """

    def get_annotation_from_xml(self, target_xml_path):
        downsamples = self.downsamples

        annotation = []
        num_annotation = 0

        tree = parse(target_xml_path)
        root = tree.getroot()
        for Annotation in root.iter("Annotation"):
            annotation_list = []
            for Coordinate in Annotation.iter("Coordinate"):
                x = round(float(Coordinate.attrib["X"]) / downsamples)
                y = round(float(Coordinate.attrib["Y"]) / downsamples)
                annotation_list.append((x, y))
            annotation.append(np.asarray(annotation_list))

        return annotation

    """
    """

    def check_path(self, dir_name):
        path = ""

        while(True):
            split = dir_name.split('/', 1)
            path = path + split[0] + '/'

            if not os.path.isdir(path):
                os.mkdir(path)
                print(path, "is created!")

            if len(split) == 1:
                break

            dir_name = split[1]

        return True

    """
    param :
    return : tissue_mask (numpy_array)
    """
    def create_tissue_mask(self, save_image=False):
        slide = self.slide
        level = self.level
        col, row = slide.level_dimensions[level]

        img = np.array(slide.read_region((0, 0), level, (col, row)))
        img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        img = img[:, :, 1]
        _, tissue_mask = cv2.threshold(img,
                                       0,
                                       255,
                                       cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        '''
        if save_image:
            target_image_path = os.path.join(self.etc_path,
                                             "tissue_mask.jpg")
            cv2.imwrite(target_image_path, tissue_mask)
        '''
        return tissue_mask


    """
    param :
    return : tumor mask (numpy_array)
    """
    def create_tumor_mask(self, save_image=False):
        slide = self.slide
        level = self.level
        annotation = self.annotation

        col, row = slide.level_dimensions[level]
        tumor_mask = np.zeros((row, col))
        cv2.drawContours(tumor_mask, annotation, -1, 255, -1)

        if save_image:
            target_image_path = os.path.join(self.etc_path,
                                             "tumor_mask.jpg")
            cv2.imwrite(target_image_path, tumor_mask)


        return tumor_mask

    """
    """
    def get_dilaero(self, mask):
        kernel_dilation = np.ones((19, 19), np.uint8)
        dilation = cv2.dilate(mask, kernel_dilation, iterations=1)
        kernel_erosion = np.ones((9, 9), np.uint8)
        erosion = cv2.erode(mask, kernel_erosion, iterations=1)
        return dilation, erosion


    """
    param : mask (numpy)
            patch_pos (tuple(x, y, width, height))
            percent (what percent will you determine as tumor)
            downsamples(int)
    return : label (int)
    """
    def determine_tumor(self, patch_pos):
        downsamples = self.downsamples
        threshold = self.threshold_of_tumor_rate
        tumor_mask = self.tumor_mask

        min_x = int(patch_pos[0] / downsamples)
        min_y = int(patch_pos[1] / downsamples)

        width = int(patch_pos[2] / downsamples)
        height = int(patch_pos[3] / downsamples)

        max_x = int(min_x + width)
        max_y = int(min_y + height)

        area = width * height

        if threshold > 1 or threshold < 0:
            raise RuntimeError('threshold must be in 0 to 1')

        #
        mask_of_patch = tumor_mask[min_y: max_y, min_x: max_x]

        #
        if np.sum(mask_of_patch) > (threshold * 255 * area):
            return 1
        else:
            return 0


    """
    param :
    return :
    """
    def get_inform_of_random_samples(self, mask, num_of_patch):
        slide = self.slide
        level = self.level
        downsamples = self.downsamples
        patch_size = self.patch_size

        set_of_inform = []
        number_of_region = int(np.sum(mask) / 255)

        if number_of_region < num_of_patch:
            raise RuntimeError(
                'Random size is bigger than number of pixels in region')

        mask = np.reshape(mask, -1)
        mask_pos = np.argwhere(mask > 0).squeeze()
        np.random.shuffle(mask_pos)
        dataset_number = mask_pos[:num_of_patch]

        width, _ = slide.level_dimensions[level]
        goleft = int(patch_size[0] / (2 * downsamples))
        goup = int(patch_size[1] / (2 * downsamples))

        for data in dataset_number:
            x = (data % width - goleft) * downsamples
            y = (data // width - goup) * downsamples

            is_tumor = self.determine_tumor(
                (x, y, patch_size[0], patch_size[1]))
            set_of_inform.append(
                [is_tumor, x, y, patch_size[0], patch_size[1]])

        return set_of_inform


    """
    param :
    return :
    """
    def get_patch_data(self, save_image=False):
        slide = self.slide
        num_of_patch = self.num_of_patch
        set_of_inform = self.set_of_inform
        set_of_patch = []

        i = 1

        if save_image:
            print("Save patch image")
            for pos in set_of_inform:
                is_tumor, x, y, w, h = pos
                patch = slide.read_region((x, y), 0, (w, h)).convert("RGB")
                set_of_patch.append(np.array(patch))

                # for image save
                patch_fn = str(x) + "_" + str(y) + "_" + str(is_tumor) + ".png"
                target_image_path = os.path.join(self.patch_path,
                                                 patch_fn)
                patch.save(target_image_path)

                print("\rPercentage : %d / %d" % (i, num_of_patch), end="")
                i = i + 1

            print("\n")
        else:
            print("Do not save patch image")
            for pos in set_of_inform:
                is_tumor, x, y, w, h = pos
                patch = slide.read_region((x, y), 0, (w, h)).convert("RGB")
                set_of_patch.append(np.array(patch))
                print("\rPercentage : %d / %d" % (i, num_of_patch), end="")
                i = i + 1
            print("\n")

        return set_of_patch


    """
    param :
    return :
    """
    def create_dataset(self, usage, slide_filename):
        set_of_patch = self.set_of_patch
        set_of_inform = self.set_of_inform

        dataset = {}

        dataset[cf.key_of_data] = np.array(set_of_patch)
        dataset[cf.key_of_informs] = np.array(set_of_inform)

        if usage == 'train':
            fp = cf.path_of_train_dataset
        elif usage == 'val':
            fp = cf.path_of_val_dataset
        elif usage == 'test':
            fp = cf.path_of_test_dataset
        else:
            raise RuntimeError("usage is invalid value")

        self.check_path(fp)

        fn = os.path.join(fp, slide_filename + ".pkl")
        fo = open(fn, 'wb')
        pickle.dump(dataset, fo, pickle.HIGHEST_PROTOCOL)
        fo.close()


    """
    param : slide file (openslide)
            level (int)
    return : thumbnail (numpy array)
    """
    def create_thumbnail(self):
        col, row = self.slide.level_dimensions[self.level]
        thumbnail = self.slide.get_thumbnail((col, row))

        thumbnail = np.array(thumbnail)
        target_image_path = os.path.join(self.etc_path, "thumbnail.jpg")
        cv2.imwrite(target_image_path, thumbnail)
        return thumbnail

    """
    param :
    use : create_thumbnail(slide, level)
    return : thumbnail (numpy array)
    """
    def draw_tumor_pos_on_thumbnail(self):
        thumbnail = self.thumbnail
        annotation = self.annotation

        cv2.drawContours(thumbnail, annotation, -1, (0, 255, 0), 4)
        target_image_path = os.path.join(self.etc_path,
                                         "tumor_to_thumbnail.jpg")
        cv2.imwrite(target_image_path, thumbnail)

        return thumbnail


    """
    brief :
    param :
    return :
    """
    def draw_patch_pos_on_thumbnail(self):
        set_of_inform = self.set_of_inform
        thumbnail = self.thumbnail
        downsamples = self.downsamples

        for inform in set_of_inform:
            is_tumor, x, y, w, h = inform
            min_x = int(x/downsamples)
            min_y = int(y/downsamples)
            max_x = min_x + int(w/downsamples)
            max_y = min_y + int(h/downsamples)

            if is_tumor:
                cv2.rectangle(thumbnail,
                              (min_x, min_y),
                              (max_x, max_y),
                              (255, 0, 0),
                              4)
            else:
                cv2.rectangle(thumbnail,
                              (min_x, min_y),
                              (max_x, max_y),
                              (0, 0, 255),
                              4)

        target_image_path = os.path.join(self.etc_path,
                                         "patch_pos_to_thumbnail.jpg")
        cv2.imwrite(target_image_path, thumbnail)

        return thumbnail


"""
"""
def prepro_use_multiprocess(usage, list_of_slide):
    print(list_of_slide)
    print(usage)
    pool = Pool(3)

    result = pool.starmap_async(
        CAMELYON_PREPRO, zip(repeat(usage), list_of_slide))

    result.wait()


"""
"""
def create_train_dataset(list_of_slide_for_train):
    print("create train dataset")
    prepro_use_multiprocess("train",
                            list_of_slide_for_train)


"""
"""
def create_val_dataset(list_of_slide_for_val):
    print("create val dataset")
    prepro_use_multiprocess("val",
                            list_of_slide_for_val)


def create_incorrect_dataset(list_of_slide_for_incorrect):
    print("creat incorrect dataset")
    prepro_use_multiprocess("train_incorrect",
                            list_of_slide_for_train)

    prepro_use_multiprocess("val_incorrect",
                            list_of_slide_for_val)


"""
"""
def create_test_dataset():
    print("create test dataset")
    CAMELYON_PREPRO("test", "test")


if __name__ == "__main__":
    start_time = time.time()

    create_train_dataset(cf.list_of_slide_for_train)
    create_val_dataset(cf.list_of_slide_for_val)
    create_incorrect_dataset(cf.list_of_slide_for_incorrect)

    end_time = time.time()
    print("Run time is :  ", end_time - start_time)
    print("Done")

create train dataset
['b_1', 'b_3', 'b_4', 'b_6', 'b_7', 'b_8', 'b_9', 'b_11', 'b_12', 'b_14']
train
allocator b_1
allocator b_4
allocator b_3
allocator b_6
allocator b_7
allocator b_8
allocator b_9
allocator b_11
allocator b_14
allocator b_12
create val dataset
['b_2', 'b_5', 'b_10', 'b_13', 'b_15']
val
allocator b_5
allocator b_10
allocator b_13
allocator b_2
allocator b_15


AttributeError: ignored

In [ ]:
from zipfile import ZipFile
file_name = "/content/drive/My Drive/CAMELYON17/training/lesion_annotations.zip"
with ZipFile(file_name,'r') as zip :
  zip.extractall()
  print('finish')

finish
